# Single perturbations

In this notebook, we explore the effects of dual-node perturbations on the behaviour of the in vivo model.

Specifically, we:
 1. Build the petri net encoding of the model.
 2. Generate knockouts for all druggable intervention combinations.
 3. Filter out non-viable interventions (`Apoptosis >= 3` in `healthy` model)
 3. Test remaining interventions on `mixed-myc-high` and `mixed-myc-low` models.
 4. Compute and compare "reliability" and "opportunity" scores.
 5. Compare distribution of opportunity and reliability scores.

In [1]:
from trapmvn.core import trapmvn
from trapmvn.representation.bma import BMA_Model
from trapmvn.representation.symbolic import Symbolic_Model
from trapmvn.representation.petri_net import Petri_Net
from pathlib import Path
import csv
import sys

MODEL_TYPE = ["healthy", "myc-low", "mixed-myc-low", "mixed-myc-high", "myc-high"]

DRUGGABLE_PROLIFERATION = Path("druggable.proliferation.txt").read_text().split(",")
DRUGGABLE_APOPTOSIS = Path("druggable.apoptosis.txt").read_text().split(",")

# These lists should be equivalent, they are just ordered differently
# to mirror the result from the original paper.
assert set(DRUGGABLE_APOPTOSIS) == set(DRUGGABLE_PROLIFERATION)
DRUGGABLE = sorted(DRUGGABLE_APOPTOSIS)

In [2]:
models = {}

for model_type in MODEL_TYPE:
    models[model_type] = BMA_Model.from_json_file(f"./models/in_vivo.{model_type}.json")
    
variables = sorted(next(iter(models.values())).variables.keys())

In [3]:
petri_nets = {}

# This should take a few seconds, since the models aren't exactly small.
for model_type, model in models.items():
    symbolic_model = Symbolic_Model.from_bma(model)
    petri_nets[model_type] = Petri_Net.build(symbolic_model, unitary=True)
    print(f"Translated {model_type}.")
    sys.stdout.flush()

Translated healthy.
Translated myc-low.
Translated mixed-myc-low.
Translated mixed-myc-high.
Translated myc-high.


In [ ]:
# Here, we compute the perturbation results for all model types.
# Note that this process can take several hours, as the number
# of interventions is very high.

intervention_results = {}
with open('dual-perturbation.tsv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    
    header = ["Intervention", "Model"] + variables
    writer.writerow(header)
    
    for i in range(len(DRUGGABLE)):
        for j in range(len(DRUGGABLE)):
            if j <= i:
                # Only test unique combinations
                continue
            intervention = f"{DRUGGABLE[i]}+{DRUGGABLE[j]}"
            intervention_results[intervention] = {}
            
            print(f"Computing ... {i+1}/{len(DRUGGABLE)}; {j+1}/{len(DRUGGABLE)}; {intervention}.")
            sys.stdout.flush()
            
            # Compute and save trap space of the wildtype model.
            wt_pn = petri_nets['healthy'].knockout(DRUGGABLE[i]).knockout(DRUGGABLE[j])
            wt_trap = trapmvn(wt_pn, None)
            assert len(wt_trap) == 1
            wt_trap = wt_trap[0]
            intervention_results[intervention]['healthy'] = wt_trap
            row = [intervention, 'healthy'] + [ str(wt_trap[x]) for x in variables ]
            writer.writerow(row)
            csvfile.flush()
            
            # If a wildtype model has worst-case `Apoptosis` value more
            # than three, we skip to the next intervention.
            if wt_trap['Apoptosis_id316'][-1] >= 3:
                print(f"Skipped as non-viable.")
                continue
            
            # Otherwise, we compute the results for the remaining two models. 
            for model_type in ["mixed-myc-low", "mixed-myc-high"]:
                pn = petri_nets[model_type].knockout(DRUGGABLE[i]).knockout(DRUGGABLE[j])
                trap = trapmvn(pn, None)
                assert len(trap) == 1
                trap = trap[0]
                intervention_results[intervention][model_type] = trap
                row = [intervention, model_type] + [ str(trap[x]) for x in variables ]
                writer.writerow(row)
                csvfile.flush()

Computing ... 1/50; 2/50; A1_id357+ARF_id177.
Computing ... 1/50; 3/50; A1_id357+Akt_id40.
Skipped as non-viable.
Computing ... 1/50; 4/50; A1_id357+BAD_id324.
Computing ... 1/50; 5/50; A1_id357+BAXBAK_id318.
Computing ... 1/50; 6/50; A1_id357+BIM_id323.
Computing ... 1/50; 7/50; A1_id357+Bcl_2_id319.
Computing ... 1/50; 8/50; A1_id357+Bcl_W_id356.
Computing ... 1/50; 9/50; A1_id357+Bcl_xl_id320.
Computing ... 1/50; 10/50; A1_id357+Beta_Catenin_id227.
Computing ... 1/50; 11/50; A1_id357+CDK2_id644.
Computing ... 1/50; 12/50; A1_id357+CDK4_id60.
Computing ... 1/50; 13/50; A1_id357+COX2_id564.
Computing ... 1/50; 15/50; A1_id357+Caspase9_id343.
Computing ... 1/50; 16/50; A1_id357+E2F_1_id199.
Computing ... 1/50; 17/50; A1_id357+EP4_id610.
Computing ... 1/50; 18/50; A1_id357+EZH2_id535.
Computing ... 1/50; 19/50; A1_id357+Elk_1_id661.
Computing ... 1/50; 20/50; A1_id357+ErbB1_id298.
Computing ... 1/50; 21/50; A1_id357+Erk_id36.
Computing ... 1/50; 22/50; A1_id357+Ets_2_id38.
Computing ...

In [23]:
# Alternatively, we can just load the results from 
# a file that we saved before.

intervention_results = {}
with open('dual-perturbation.tsv') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    next(reader) # Skip header
    for row in reader:
        intervention = row[0]
        model_type = row[1]
        trap = { var: eval(row[i+2]) for i, var in enumerate(variables) }
        if intervention not in intervention_results:
            intervention_results[intervention] = {}
        intervention_results[intervention][model_type] = trap

In [39]:
# Now we can filter out interventions which are not
# viable due to high apoptosis values:

print(f"Before filtering: {len(intervention_results)}")
interventions = list(intervention_results.keys())
for intervention in interventions:
    if len(intervention_results[intervention]) == 1:
        del intervention_results[intervention]
    # We "skip" the myc-related interventions because
    # it is not clear how they are implemented across
    # different model variants.
    if "Myc" in intervention:
        del intervention_results[intervention]
print(f"After filtering: {len(intervention_results)}")
interventions = list(intervention_results.keys())

Before filtering: 1040
After filtering: 995


In [40]:
# Finally, we can compute the reliability and opportunity
# scores for all perturbations:
opportunity = {}
reliability = {}

for intervention in interventions:
    x = intervention_results[intervention]
    a_score = x['mixed-myc-low']['Apoptosis_id316'][-1] + x['mixed-myc-high']['Apoptosis_id316'][-1]
    p_score = x['mixed-myc-low']['Proliferation_id47'][0] + x['mixed-myc-high']['Proliferation_id47'][0]
    opportunity[intervention] = a_score - p_score
    
    a_score = x['mixed-myc-low']['Apoptosis_id316'][0] + x['mixed-myc-high']['Apoptosis_id316'][0]
    p_score = x['mixed-myc-low']['Proliferation_id47'][-1] + x['mixed-myc-high']['Proliferation_id47'][-1]
    reliability[intervention] = a_score - p_score

In [41]:
# We then sort the interventions by the average of the two
# scores and output the result to a file.

sorted_interventions = sorted(interventions, key=lambda x: -(opportunity[x] + reliability[x])/2)
with open('sorted-dual-perturbation.tsv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    
    header = ["Intervention", "Reliability", "Opportunity"]
    header += ["Apoptosis (WT)", "Apoptosis (mixed-myc-low)", "Apoptosis (mixed-myc-high)"]
    header += ["Proliferation (mixed-myc-low)", "Proliferation (mixed-myc-high)"]
    writer.writerow(header)
    for i in sorted_interventions:    
        row = [i, reliability[i], opportunity[i]]
        row += [intervention_results[i][m]['Apoptosis_id316'] for m in ['healthy', 'mixed-myc-low', 'mixed-myc-high']]
        row += [intervention_results[i][m]['Proliferation_id47'] for m in ['mixed-myc-low', 'mixed-myc-high']]
        writer.writerow(row)

In [46]:
# Finally, we are interested in the distribution of 
# score differences.

differences = {}
for i in sorted_interventions:
    diff = opportunity[i] - reliability[i]
    if diff not in differences:
        differences[diff] = 0
    differences[diff] += 1
    
differences_histogram = [(k, differences[k]) for k in sorted(differences.keys())]

In [47]:
differences_histogram

[(0, 439),
 (1, 17),
 (2, 76),
 (3, 160),
 (4, 69),
 (5, 57),
 (6, 45),
 (7, 29),
 (8, 49),
 (9, 48),
 (10, 4),
 (11, 2)]